In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
ACCOUNT_KEY = "<your key>"
STORAGE_ACCT_NAME = "sparkpocdatastorage"
CONTAINER_NAME = "csvdatastore"

In [3]:
options = {'account_key':ACCOUNT_KEY}

In [4]:
df = pd.read_csv('abfs://'+CONTAINER_NAME+'@'+STORAGE_ACCT_NAME+'.dfs.core.windows.net/heart_data.csv', storage_options=options)

In [5]:
df.head(5)

,index,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [6]:
spark = SparkSession.builder.appName("sparkdev-adlfs-integration").master("local[*]").getOrCreate()

23/04/20 19:55:31 WARN Utils: Your hostname, Pavans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.29.143 instead (on interface en0)
23/04/20 19:55:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/20 19:55:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark_df = spark.createDataFrame(df)

/Users/pavanmantha/miniconda3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/Users/pavanmantha/miniconda3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [8]:
spark_df.show(5, truncate=False)

+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|index|id |age  |gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|0    |0  |18393|2     |168   |62.0  |110  |80   |1          |1   |0    |0   |1     |0     |
|1    |1  |20228|1     |156   |85.0  |140  |90   |3          |1   |0    |0   |1     |1     |
|2    |2  |18857|1     |165   |64.0  |130  |70   |3          |1   |0    |0   |0     |1     |
|3    |3  |17623|2     |169   |82.0  |150  |100  |1          |1   |0    |0   |1     |1     |
|4    |4  |17474|1     |156   |56.0  |100  |60   |1          |1   |0    |0   |0     |0     |
+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
only showing top 5 rows



In [25]:
# Find Count of Null, None, NaN of All DataFrame Columns
spark_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----+---+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|index| id|age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+-----+---+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|    0|  0|  0|     0|     0|     0|    0|    0|          0|   0|    0|   0|     0|     0|
+-----+---+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+



In [9]:
columns = spark_df.columns

In [10]:
columns.remove('id')
columns.remove('index')

In [11]:
input_columns = columns[:11]
target_column = columns[:-1]

In [12]:
input_columns

['age',
 'gender',
 'height',
 'weight',
 'ap_hi',
 'ap_lo',
 'cholesterol',
 'gluc',
 'smoke',
 'alco',
 'active']

In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
vector_assembler = VectorAssembler(inputCols=input_columns, outputCol="features", handleInvalid="skip")
transformed_df = vector_assembler.transform(spark_df)
transformed_df.show(5, truncate=False)

+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+--------------------------------------------------------+
|index|id |age  |gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|features                                                |
+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+--------------------------------------------------------+
|0    |0  |18393|2     |168   |62.0  |110  |80   |1          |1   |0    |0   |1     |0     |[18393.0,2.0,168.0,62.0,110.0,80.0,1.0,1.0,0.0,0.0,1.0] |
|1    |1  |20228|1     |156   |85.0  |140  |90   |3          |1   |0    |0   |1     |1     |[20228.0,1.0,156.0,85.0,140.0,90.0,3.0,1.0,0.0,0.0,1.0] |
|2    |2  |18857|1     |165   |64.0  |130  |70   |3          |1   |0    |0   |0     |1     |[18857.0,1.0,165.0,64.0,130.0,70.0,3.0,1.0,0.0,0.0,0.0] |
|3    |3  |17623|2     |169   |82.0  |150  |100  |1          |1   |0    |0   |1     |1     |[17623.0

In [15]:
required_df = transformed_df.drop('index', 'id', 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active')
required_df.show(5, truncate=False)

+------+--------------------------------------------------------+
|cardio|features                                                |
+------+--------------------------------------------------------+
|0     |[18393.0,2.0,168.0,62.0,110.0,80.0,1.0,1.0,0.0,0.0,1.0] |
|1     |[20228.0,1.0,156.0,85.0,140.0,90.0,3.0,1.0,0.0,0.0,1.0] |
|1     |[18857.0,1.0,165.0,64.0,130.0,70.0,3.0,1.0,0.0,0.0,0.0] |
|1     |[17623.0,2.0,169.0,82.0,150.0,100.0,1.0,1.0,0.0,0.0,1.0]|
|0     |[17474.0,1.0,156.0,56.0,100.0,60.0,1.0,1.0,0.0,0.0,0.0] |
+------+--------------------------------------------------------+
only showing top 5 rows



In [16]:
required_df.select(col("cardio")).distinct().show(truncate=False)

+------+
|cardio|
+------+
|0     |
|1     |
+------+



In [17]:
(train_set, test_set) = required_df.randomSplit([0.8,0.2])

In [18]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [19]:
dt = DecisionTreeClassifier(featuresCol='features', labelCol='cardio')
model = dt.fit(train_set)

In [20]:
predictions = model.transform(test_set)
predictions.show(10, truncate=False)

+------+-------------------------------------------------------+----------------+----------------------------------------+----------+
|cardio|features                                               |rawPrediction   |probability                             |prediction|
+------+-------------------------------------------------------+----------------+----------------------------------------+----------+
|0     |[14398.0,2.0,177.0,82.0,110.0,80.0,1.0,1.0,1.0,1.0,1.0]|[14810.0,4151.0]|[0.7810769474183851,0.2189230525816149] |0.0       |
|0     |[14405.0,1.0,160.0,74.0,100.0,60.0,1.0,1.0,0.0,0.0,1.0]|[14810.0,4151.0]|[0.7810769474183851,0.2189230525816149] |0.0       |
|0     |[14416.0,1.0,155.0,68.0,140.0,90.0,3.0,3.0,0.0,0.0,1.0]|[2484.0,13022.0]|[0.16019605314071972,0.8398039468592803]|1.0       |
|0     |[14418.0,2.0,168.0,72.0,120.0,80.0,1.0,1.0,0.0,0.0,1.0]|[14810.0,4151.0]|[0.7810769474183851,0.2189230525816149] |0.0       |
|0     |[14425.0,1.0,168.0,63.0,909.0,60.0,2.0,1.0,0.0,0.0,1.0

In [21]:
evaluator = MulticlassClassificationEvaluator(labelCol='cardio', predictionCol='prediction')
accuracy = evaluator.evaluate(predictions)
print('Accuracy=', accuracy)
print('Test Error=', 1.0-accuracy)

Accuracy= 0.7210138597018063
Test Error= 0.2789861402981937


In [22]:
rfc = RandomForestClassifier(featuresCol='features',labelCol='cardio', numTrees=5)
model_rfc = rfc.fit(train_set)

In [23]:
predictions_rfc = model_rfc.transform(test_set)
predictions_rfc.show(5, truncate=False)

+------+-------------------------------------------------------+---------------------------------------+----------------------------------------+----------+
|cardio|features                                               |rawPrediction                          |probability                             |prediction|
+------+-------------------------------------------------------+---------------------------------------+----------------------------------------+----------+
|0     |[14398.0,2.0,177.0,82.0,110.0,80.0,1.0,1.0,1.0,1.0,1.0]|[3.733422596196178,1.2665774038038218] |[0.7466845192392355,0.2533154807607644] |0.0       |
|0     |[14405.0,1.0,160.0,74.0,100.0,60.0,1.0,1.0,0.0,0.0,1.0]|[3.770127284184377,1.2298727158156226] |[0.7540254568368754,0.24597454316312453]|0.0       |
|0     |[14416.0,1.0,155.0,68.0,140.0,90.0,3.0,3.0,0.0,0.0,1.0]|[0.9208302858006626,4.079169714199337] |[0.1841660571601325,0.8158339428398674] |1.0       |
|0     |[14418.0,2.0,168.0,72.0,120.0,80.0,1.0,1.0,0.0,0.0

In [24]:
evaluator = MulticlassClassificationEvaluator(labelCol='cardio', predictionCol='prediction')
accuracy = evaluator.evaluate(predictions_rfc)
print('Accuracy=', accuracy)
print('Test Error=', 1.0-accuracy)

Accuracy= 0.7221677300394725
Test Error= 0.27783226996052746
